In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import string
import nltk
from nltk import pos_tag 
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
from nltk.stem.isri import ISRIStemmer
import geocoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import cluster
import ast
from datetime import date
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as mc
import math
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import cluster
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
def read_data(path):
    data=pd.read_excel(path)
    df = data.rename(columns=lambda x: x.strip())
    return df

In [3]:
def concat_all(data,axis):
    main_df=pd.concat(data,sort=True,axis=axis)
    main_df=main_df.reset_index().drop(["index"],axis=1)
    return main_df

In [4]:
def drop_columns(data):
    full_null=data.columns[data.isna().sum()>=(len(data)-5)].tolist()
    droped_coulmns=(full_null+["old collector"]+["New collector"]+["Customer Name"]
                    +["Agency Name"]+["حساب احتياطى"]+["Region"]+['Branch name']
                    +["ADDRESS1","ADDRESS2","ADDRESS3"]+['التعليق']+["NationalID"]
                    +['JOINT Gurantor_CLI']+["Average amount should be paid to Mov ODA 1 bucket WITHIN same month"]
                    +["data"]+["Home"]+["Work"]+['التاريخ في حاله السداد','التاريخ في حاله الوعد بالسداد']+["نتيجه الاتصال","نوع الاتصال"]
                    +['latitude']+['longitude']+['Mobile','comment','legal feedback'])
   
    data=data.drop(columns=droped_coulmns,axis=1)
    return data 

In [5]:
def fill_na(data):
    x=lambda x: 0 if type(x)==str else x
    data['المبلغ']=data['المبلغ'].fillna(0)
    data["Age"]=data["Age"].fillna(data["Age"].mean()).astype(int)
    data["Contra Dues"]=data["Contra Dues"].apply(x).fillna(0)
    data["PMT 11"]=data["PMT 11"].apply(x).fillna(0).astype(int)
    data["ptp"]=data["ptp"].apply(x).fillna(0).astype(int)
    return data

In [6]:
def NationalID(ID):
    ID=str(ID)
    today = date.today()
    if (len(ID)>=14) and (ID[0]=="2"):
        age=today.year-int("19"+ID[1:3])
    elif(len(ID)>=14) and (ID[0]=="3"):
        age=today.year-int("20"+ID[1:3])
    else:
        age=None
    return age

def get_age(series):
    age = series.apply(lambda x:NationalID(x))
    return age

In [7]:
def clean_text(text):
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [word for word in text if not any(c.isdigit() for c in word)]
    text = [t for t in text if len(t) > 0]
    text = [ISRIStemmer().stem(t) for t in text]
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    sent = re.sub("^\d+\s|\s\d+\s|\s\d+$", "", text)
    sent = re.sub("[إأآا]", "ا", sent)
    sent = re.sub("[ة]", "ه", sent)
    sent = re.sub(r'(.)\1+', r'\1', sent)
    return(text)

In [8]:
def train_vectorizer(data):
    vect = CountVectorizer().fit(data)
    return vect


In [9]:
def vectorize(data,vect):
    data_vectorized = vect.transform(data)
    return data_vectorized

In [10]:
def encode_text(series):
    cleaned_series = series.apply(lambda x :clean_text(x))
    vect = train_vectorizer(cleaned_series)
    vectorized = vectorize(cleaned_series,vect)
    vectorized = pd.DataFrame(vectorized.A)
    vectorized.columns = ['feature_vector' + str(i) for i in vectorized.columns]
    return vectorized

In [11]:
def clean_address(text):
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [t for t in text if len(t) > 0]
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    text = text + ", EGYPT"
    return(text)

In [12]:
def get_lat_long(series):
    cleaned_address = series.apply(lambda x : clean_address(x))
    get_lat_long=lambda x: geocoder.bing(x,key='Auqv-r3CfIPHqEUrZv_mp3E6dngQJc7K5Maq8moOlqHH1pdKuI-IaCp2IcoCdZ_o').latlng
    lat_long = cleaned_address.apply(get_lat_long)
    get_long = lambda x:x[1]
    get_lat = lambda x:x[0]
    latitude = lat_long.apply(get_lat)
    longitude = lat_long.apply(get_long)
    return latitude, longitude

In [13]:
def cluster_locations(lat,long,k):
    km = cluster.KMeans(n_clusters=k)
    labels = km.fit_predict(lat.values.reshape(-1,1),long.values.reshape(-1,1))
    return labels

In [14]:
#label encoding for categorical features
def label_encode_categories(data):
    sr=lambda x : str(x) if type(x)!=str else x
    for i in data.columns:
        if data[i].dtype == 'object':
            data[i]=data[i].apply(sr)
            LB = LabelEncoder()
            data[i] = LB.fit_transform(data[i].values.reshape(-1,1))
    return data


In [15]:
def mobile(number):
    number=str(number)
    if len(number)==10:
        if (number[0:2]=="11"):
            comp="Etisalat"
        elif (number[0:2]=="10"):
            comp="voda"
        elif number[0:2]=="12":
            comp="Orang"
        else:
            comp=None
    elif len(number)==11:
        if (number[0:3]=="11"):
            comp="Etisalat"
        elif (number[0:3]=="10"):
            comp="voda"
        elif number[0:3]=="12":
            comp="Orang"
        else:
            comp=None
    else:
        comp=None
    
    return comp

def get_mobile_co(serios):
    
    co=serios.apply(lambda x: mobile(x))
    
    return co
    

In [16]:
def preprocessing(feature,k,axis):
    feature["Age"]=get_age(feature.NationalID)
    feature['latitude'],feature['longitude']=get_lat_long(feature["JOINT Gurantor_CLI"])
    feature["location_cluster"]=cluster_locations(feature['latitude'],feature['longitude'],k)
    CT_vect=encode_text(feature["التعليق"])
    feature=concat_all([feature,CT_vect],axis)
    feature["Mobile_co"]=get_mobile_co(feature.Mobile)
    feature=fill_na(feature)
    feature=drop_columns(feature)
    feature=label_encode_categories(feature)
    return feature
    

In [17]:
sara = read_data('ساره.xlsx')

In [18]:
marina = read_data("مارينا.xlsx")

In [19]:
data=concat_all([marina,sara],0)

In [56]:
feature=data.copy()

#### remark_na=feature[feature["remark"].notnull()].copy().reset_index().drop(["index"],axis=1)

#### remark_na=remark_na.reset_index().drop(["index"],axis=1)

remark_na=preprocessing(feature[feature["remark"].notnull()].copy().reset_index().drop(["index"],axis=1),5,1)

## Fill Null for Remark Column Using Knn Algoritm

In [57]:
orignal=feature["remark"]

In [58]:
orignal.value_counts()

ptp        12
pay        11
follow      6
refuse      5
pay+ptp     2
Name: remark, dtype: int64

In [59]:
mask=feature["remark"].isna()

In [60]:
mask

0      True
1      True
2      True
3      True
4      True
       ... 
630    True
631    True
632    True
633    True
634    True
Name: remark, Length: 635, dtype: bool

In [61]:
feature=preprocessing(feature.drop(columns=["CID"],axis=1),200,1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [74]:
feature.remark.value_counts()

4    12
2    11
0     6
5     5
3     2
Name: remark, dtype: int64

In [63]:
feature["remark"]=feature["remark"].where(~mask,orignal)

In [87]:
X_RK=feature[feature["remark"].notna()].drop(columns=["remark"])
Y_Rk=feature["remark"][feature["remark"].notna()].values.ravel().astype("int")

In [88]:
scale=MinMaxScaler()
X= scale.fit_transform(X_RK)

In [169]:
knn = KNeighborsClassifier( n_neighbors=1)

In [170]:
train=knn.fit(X,Y_Rk.ravel())

In [171]:
test=feature[feature["remark"].isna()].drop(columns=["remark"])

In [172]:
test=scale.fit_transform(feature[feature["remark"].isna()].drop(columns=["remark"]))

In [173]:
X_RK.shape

(36, 277)

In [174]:
test.shape

(599, 277)

In [175]:
Y_Rk

array([2, 0, 5, 0, 0, 2, 4, 0, 4, 2, 4, 0, 5, 0, 2, 4, 4, 4, 5, 3, 4, 4,
       5, 4, 4, 3, 2, 5, 4, 2, 2, 2, 4, 2, 2, 2])

In [176]:
predict=train.predict(test)

In [177]:
dff=pd.DataFrame(predict)

In [178]:
dff[0].value_counts()

2    573
5     17
4      9
Name: 0, dtype: int64

In [179]:
df_out = pd.merge(feature[feature["remark"].isna()].reset_index(),dff,how = 'left',left_index = True, right_index = True)

In [180]:
df_out=df_out.set_index(df_out["index"]).drop(columns=["index"])

In [181]:
df_out[['remark',0]]

,remark,0
index,,
0,NaN,2
1,NaN,2
2,NaN,2
3,NaN,2
4,NaN,2
...,...,...
630,NaN,2
631,NaN,2
632,NaN,2


In [182]:
feature["remark"]=feature["remark"].fillna(df_out[0])